#Подключение google-disk

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Загрузка библиотек

In [ ]:
import re
import time
import copy

import pandas as pd
import numpy as np
import scipy
import pyarrow.parquet as pq

import joblib
from joblib import Parallel, delayed

import nltk
from nltk.probability import FreqDist

from collections import Counter

from sklearn.preprocessing import LabelBinarizer, StandardScaler, OrdinalEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.compose import ColumnTransformer
from sklearn.metrics  import f1_score, roc_auc_score
from lightgbm import LGBMClassifier, LGBMRegressor
from sklearn.base import BaseEstimator, TransformerMixin

from tqdm import notebook
from tqdm.auto import tqdm

## Определение пользовательских функций

In [ ]:
#Функция, которая ищет наиболее часто встречающиеся сайты (количество в вывводе = numb_commons) из выборки
def search_top_urls(strat_data, numb_commons):
    urls = " ".join(strat_data.url_cat)
    tokens = urls.split()
    text = nltk.Text(tokens)
    fdist = FreqDist(text)
    return fdist.most_common(numb_commons)


In [ ]:
#Функция, которая ищет среди наиболее популярных сайтов те, которые находяться вне пересения множеств сайтов для мужчин и женщин 
def search_commons(data, all_top_amt, rel_top_amt):

    top_m_list = search_top_urls(data.query('is_male == 1'), all_top_amt)
    top_w_list = search_top_urls(data.query('is_male == 0'), all_top_amt)

    top_m = {pair[0] for pair in top_m_list}
    top_w = {pair[0] for pair in top_w_list}

    diff_for_m = top_m.difference(top_w)
    diff_for_w = top_w.difference(top_m)
    
    top_for_m = [pair[0] for pair in top_m_list if pair[0] in diff_for_m][:rel_top_amt]
    top_for_w = [pair[0] for pair in top_w_list if pair[0] in diff_for_w][:rel_top_amt]
    
    print(top_for_m)
    print(top_for_w)

    return top_for_m + top_for_w

In [ ]:
#Функция, которая ищет среди наиболее популярных сайтов те, которые находяться вне пересения множеств сайтов для разных возрастных групп 
def search_commons_age(data, all_top_amt, rel_top_amt):
    tops = []
    tqdm.pandas(desc="Calc relevant url-s progress ...")
    for idx in notebook.tqdm(range(1,8)):
        klass_list = list(range(1,8))
        klass_list.remove(idx)

        top_lists = [search_top_urls(all_data.query('klass == @klas'), all_top_amt//10) for klas in klass_list]
        top = search_top_urls(all_data.query('klass == @idx'), all_top_amt)

        top_sets = [{pair[0] for pair in top_list} for top_list in top_lists] 
        top_set = {pair[0] for pair in top}
    
        all_set = top_sets[0].union(top_sets[1], top_sets[2], top_sets[3], top_sets[4], top_sets[5])

        diff = top_set.difference(all_set)
  
        rez_top = [pair[0] for pair in top if pair[0] in diff][:rel_top_amt]
        tops.extend(rez_top)
        print(rez_top)
    return tops

In [ ]:
#Функция, которая разделяет записи на возрастные группы
def my_class(age):
    if (age < 26 and age > 18):
        return 1
    if (age < 36 and age > 25):
        return 2
    if (age < 46 and age > 35):
        return 3
    if (age < 56 and age > 45):
        return 4
    if (age < 66 and age > 55):
        return 5
    if (age > 66):
        return 6
    else:
        return 7 

In [ ]:
#Разделяет записи на возрастные группы
def my_class_2(age):
    if (age < 26):
        return 1
    if (age <= 35 and age >= 26):
        return 2
    if (age <= 45 and age >= 36):
        return 3
    if (age <= 55 and age >= 46):
        return 4
    if (age <= 65 and age >= 56):
        return 5
    if (age > 65):
        return 6
    else:
        return 7 


In [ ]:
#Функция, которая оставляет в списке адресов только те, которые есть в закрытой выборке
def search_closed(url_host, closed_list):
    old_list = str(url_host).split()
    new_list = [url for url in old_list if url in closed_list]
    return (" ".join(new_list)).strip()

In [ ]:
#Поиск лучших гиперпараметров для задачи классификации на мужчин/женщих
def search_best_params_male(how, X, y_true, model_obj, grid_params, STOP_EN, cv_numb = 3):
    if how == 'no_tf':
         pipe = Pipeline([                    
                        ('model', model_obj)
                    ])
    else:
        pipe = Pipeline([                    
                    ('tfidf', TfidfVectorizer()),
                    ('pca', TruncatedSVD(random_state = 42)), 
                    ('model', model_obj)
                    ])
    
    grid_obj = GridSearchCV(\
    estimator=pipe, param_grid=grid_params, n_jobs=1, scoring='roc_auc', cv = cv_numb, verbose=2) 
    
    print(f'START fiting in {time.strftime("%H:%M:%S", time.localtime())}')
    start = time.time()
    grid_obj.fit(X, y_true)
    end = time.time()
    grid_time = (end - start)
   
    print(f'Grid searching finished. \
    roc_auc: {grid_obj.best_score_},\
    Grid time: {grid_time} s,\
    Fit time: {np.mean(grid_obj.cv_results_.get("mean_fit_time"))} s,\
    Predict time: {np.mean(grid_obj.cv_results_.get("mean_score_time"))} s')

    times = (grid_time, np.mean(grid_obj.cv_results_.get('mean_fit_time')), \
             np.mean(grid_obj.cv_results_.get('mean_score_time')))

    #Выводим лучшие гиперпараметры
    try:
        print(f'best params: {grid_obj.best_params_}')
    except:
        print(f'проблемы с поиском лучшей модели')
     
    return (grid_obj, times)  

In [ ]:
#Поиск лучших гиперпараметров для задачи расчета возраста
def search_best_params_age(how, X, y_true, model_obj, grid_params, STOP_EN, cv_numb = 3):
    if how == 'no_tf':
         pipe = Pipeline([                    
                        ('model', model_obj)
                    ])
    else:
        pipe = Pipeline([('tfidf', TfidfVectorizer()),
                         ('pca', TruncatedSVD(random_state = 42)), 
                         ('model', model_obj)])
    
    grid_obj = GridSearchCV(estimator=pipe, param_grid=grid_params, n_jobs=-1, scoring='neg_mean_squared_error', cv=cv_numb, verbose=2) 
    
    print(f'START fiting in {time.strftime("%H:%M:%S", time.localtime())}')
    start = time.time()
    grid_obj.fit(X, y_true)
    end = time.time()
    grid_time = (end - start)
   
    print(f'Grid searching finished. \
    F1: {grid_obj.best_score_},\
    Grid time: {grid_time} s,\
    Fit time: {np.mean(grid_obj.cv_results_.get("mean_fit_time"))} s,\
    Predict time: {np.mean(grid_obj.cv_results_.get("mean_score_time"))} s')

    times = (grid_time, np.mean(grid_obj.cv_results_.get('mean_fit_time')), \
             np.mean(grid_obj.cv_results_.get('mean_score_time')))

    #Выводим лучшие гиперпараметры
    try:
        print(f'best params: {grid_obj.best_params_}')
    except:
        print(f'проблемы с поиском лучшей модели')
     
    return (grid_obj, times)  

In [ ]:
#Функция, которая удаляет из url то, что после последней точки (доменную зону)
def clean_end_of_urls(url_string):
    str_list = url_string.split()
    str_list = [one_str.rsplit('.', 1)[0] for one_str in str_list]
    new_str = str_list
    return " ".join(new_str)

In [ ]:
#Функция, которая удаляет все кроме основного домена 
def clean_url_domen(url_string):
  str_list = url_string.split()
  str_list = [one_str.rsplit('.', 1)[0] for one_str in str_list]
  new_str = []
  for one_str in str_list:
    try:
      new_str.append(one_str.rsplit('.', 1)[1])
    except:
      new_str.append(one_str.rsplit('.', 1)[0])
    
  return " ".join(new_str)

In [ ]:
#Функция, которая отчищает данные из списка категорий до основного домена
def clean_url_catigories(url_string):
  url_string = url_string.rsplit('://', 1)[1]
  url_string = url_string.split('/', 1)[0]
  url_string = url_string.rsplit('.', 1)[0]
  if url_string.rsplit('.', 1)[0] == 'www':
    url_string = url_string[4:]
  return url_string

In [ ]:
#Функция, которая расчитывает словарь (основной домен: категория)
def calc_url_and_cat_dict(df_site_cat):
  df_site_cat['cat'] = df_site_cat['Подр. 3']
  null_index = df_site_cat.query('cat.isna()').index

  for one_index in null_index:
    df_site_cat.iloc[one_index, 15] = df_site_cat.iloc[one_index, 4]

  null_index = df_site_cat.query('cat.isna()').index
  for one_index in null_index:
    df_site_cat.iloc[one_index, 15] = df_site_cat.iloc[one_index, 3]

  domen_list = list(df_site_cat['site'])
  cat_list = list(df_site_cat['cat'])
  domen_cat = {domen_list[ind]: str(cat_list[ind]) for ind in range(len(domen_list))}
  return domen_cat

In [ ]:
#Функция которая подбирает под каждый сайт пользователя категорию или (если ее нет) - пустоту ("")
def my_search(df):
  domen_list = df['clean_url_host'].split()
  url_list = df.url_host.split()
 
  str_list = [domen_cat.get(str(one_str), "") for ind, one_str in enumerate(domen_list)]
  try:
    string = (" ".join(str_list)).strip()
  except:
    print(str_list)
  return " ".join(str_list)

In [ ]:
#Класс - костыль, чтобы можно было подавать на вход tf-idf объекта только один столбец с URL
class ColumnSelector(BaseEstimator, TransformerMixin):
    """Select only specified columns."""
    def __init__(self, columns):
        self.columns = columns
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        print(type(X[self.columns]))
        return X[self.columns]

In [ ]:
#Константные списки
part_days = ['evening', 'night', 'day', 'morning']
week_days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday','Friday', 'Saturday', 'Sunday' ]

## Загрузка данных

In [ ]:
#Загрузка ДФ с категориями
try:
  site = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/yaca_base.xlsx")
except:
  site = pd.read_excel("yaca_base.xlsx")

#Все признаки (open and closed)
try:
  all_users = pd.read_csv("/content/drive/MyDrive/all_users_prepared")
except:
  all_users = pd.read_csv("all_data_prepared_2")

#Данные с целевым признаком (age and is_male)
try:
  target_data = pd.read_parquet("/content/drive/MyDrive/public_train.pqt")
except:
  target_data = pd.read_parquet("public_train.pqt")

#Список идентификаторов пользователей, по которым надо сделать предсказание
try:
  id_to_submit = pq.read_table("/content/drive/MyDrive/submit_2.pqt").to_pandas()
except:
  id_to_submit = pq.read_table("submit_2.pqt").to_pandas()

In [ ]:
all_users = all_users.drop('Unnamed: 0', axis = 1, errors = 'ignore')

In [ ]:
catigory_cols = ['name:city', 'name:region', 'name:cpe', 'name: cpe_type_cd']

# Скромная предъобработка

In [ ]:
# Изменение типов данных (для дат)
all_users['begin_date'] = pd.to_datetime(all_users['begin_date'])
all_users['end_date'] = pd.to_datetime(all_users['end_date'])

In [ ]:
# Проверка на пропуски
all_users.url_host.isna().sum()

0

## Отчистка url

In [ ]:
#Удаляем доменные зоны
all_users.url_host = all_users.url_host.transform(lambda x: clean_end_of_urls(str(x)))

#Убираем цифры
p2 = re.compile(r'\d+[.-]*')
all_users.url_host = all_users.url_host.transform(lambda x: p2.sub('', str(x)))

#Убираем nuserapi (ели она повторяется в url-адресе больше одного раза)
p3 = re.compile(r'(nuserapi)(\1{1,})')
repl = 'nuserapi'
all_users.url_host = all_users.url_host.transform(lambda x: re.sub(p3, repl, str(x)))

#Проверка результата
all_users.url_host.head(5)

0    kp yastatic avatars.mds.yandex sunuserapi yhb....
1    avatars.mds.yandex avatars.mds.yandex zen.yand...
2    avatars.mds.yandex avatars.mds.yandex relap sm...
3    ad.mail ads.betweendigital go.mail googleads.g...
4    ad.mail avatars.mds.yandex cdn-rtb.sape i.ytim...
Name: url_host, dtype: object

## Расчет статистик по категориям

Для того, чтобы снизить размерность признаков принято решение попробовать:
- взять общедоступный датафрейм, в котором храняться записи с названиями сайтов и их категориями с несколькими уровнями конкретизации
- заменить названия сайтов в анализируемом ДФ на названия категорий из дополнительного ДФ.

Идея такая: сайтов с названиями kolesa, koleso, 4kolesa много, но все они относяться к одной категории - автотовары. Соответсвенно, такая замена возможно поможет модели легче подбирать для разных групп людей релевантные категории.

In [ ]:
#Заполняем пустые url_host 
all_users.url_host = all_users.url_host.fillna('no')

#Чистим urls до домена
all_users['clean_url_host'] = all_users.url_host.transform(lambda x: clean_url_domen(str(x)))

#Чистим urls из списка категорий до домена
site['site'] = site['Сайт'].transform(lambda x: clean_url_catigories(str(x)))

#Словарь домен: категория
global domen_cat
domen_cat = calc_url_and_cat_dict(site)

#Расчитываем для каждого пользователя все его категории
all_users['url_cat'] = all_users.apply(my_search, axis = 1)
all_users['url_cat'].head(5)

0    Eurovision  Broadcasts     Email  Email   Othe...
1    Broadcasts Broadcasts  Email MobileApps    Bro...
2    Broadcasts Broadcasts  Infrastructure Search_E...
3         General  Building_Supplies Search_Engines...
4     Broadcasts  Email  Email    Search_Engines   ...
Name: url_cat, dtype: object

In [ ]:
#Расчитываем все имеющиеся в исследуемом ДФ уникальные категории
all_uniq_cats = set((" ".join(all_users.url_cat)).split())
all_uniq_cats

{'12545',
 '3D',
 '3D-shooters',
 '4651',
 '4760',
 'Actors',
 'Adult',
 'Adventures',
 'Advertizing',
 'Agency',
 'Alcohol',
 'Alternative',
 'Amateur_photographer',
 'American',
 'American_Cars',
 'Analitics',
 'Animation',
 'Aphorismes',
 'Apple',
 'Aquarium',
 'Arcades',
 'Architecture',
 'Art_for_sale',
 'Articles',
 'Astrology',
 'Audio',
 'Audit',
 'Auto_price',
 'Automation',
 'Autoschools',
 'Autotravel',
 'Aviation',
 'Banks',
 'Banners_Networks',
 'Bards',
 'Bars',
 'Beach',
 'Beauty_salon',
 'Beer',
 'Belarusian_Literature',
 'Biographics',
 'Birds',
 'Boards',
 'Boats',
 'Bookstore',
 'Bowling',
 'Broadcasts',
 'Brokers',
 'Building_Equipment',
 'Building_Supplies',
 'Business',
 'Business_Contacts',
 'Bystrovozvodimye',
 'Car_Navigators',
 'Caricatures',
 'Catalogues',
 'Cats',
 'Celebrations',
 'Cellular',
 'Channels',
 'Chanson',
 'Cheats_and_Hints_and_Codes',
 'Chemical_Industry',
 'Chemicals',
 'Children',
 'Chinese',
 'Chinese_Cars',
 'Cinema',
 'Circus',
 'City_phot

In [ ]:
#Расчет для каждого пользователя и для каждой из имеющихся в иссдедуемом ДФ категории поличества посещений сайтов этот категории.
#Добавление в ДФ столбца для каждой категории. 
for url in tqdm(all_uniq_cats): 
 all_users[url] = all_users.url_cat.transform(lambda x: Counter(str(x).split())[url])

В результате предвичной предобработки выполнено следующее:
- заменены на datetime типы данных в столбцах с датами
- удалены доменные зоны в названиях сайтов
- удалены цыфры в названиях сайтов
- удалена многократноповторющаяся строка nuserapi
- сформирован словарь соответствий {название сайта: категория сайта}
- всех встречающихся в ДФ категорий созданы и записаны в ДФ столбцы с количеством посещений этой категории сайтов пользователем

## Подготовка для ML. Кодирование

Тепербь уже понятно, стратегия самостоятельного кодирования - это точно не лучщий вариант, если используются бустинговые модели LightGBM или CatBoost. Но момент соревнования и разработки блокнота было сделано именно так. Наиболее вероятно это в хужшую сторону повлияло на качестве предсказания. 

In [ ]:
#Coding for cpe-type
coder = LabelBinarizer()
coder.fit(all_users['cpe_type_cd'])

##Coding for all_users
coded = coder.transform(all_users['cpe_type_cd'])
coded_df = pd.DataFrame(coded, index = all_users.index)
all_users = pd.concat([all_users, coded_df], axis=1).drop(['cpe_type_cd'], axis=1)


In [ ]:
#Coding for city, region and model of cpe
encoder = OrdinalEncoder(handle_unknown = 'use_encoded_value')
encoder.set_params(unknown_value=-1)

##Coding for train
all_users.loc[:,['city', 'region', 'cpe']] =\
encoder.fit_transform(all_users[['city', 'region', 'cpe']])

## Масштабирование части данных

Так как модели линейной регрессии и им подобные не использовались, необходимость масштабирования  отсутствовала

In [ ]:
all_users.sample(5)

,user_id,begin_date,end_date,url_host,price,evening,night,day,morning,Monday,...,city_count,region_count,cpe_count,clean_url_host,url_cat,0,1,2,3,4
261849,128172,2021-06-16,2021-07-09,i.ytimg img.samsungapps sunuserapi i.ytimg i.y...,20864.0,172,46,191,92,57,...,1,1,1,i img sunuserapi i i sunuserapi sunuserapi g v...,Email Email Email Other Email Other S...,0,0,0,1,0
73422,317999,2021-06-16,2021-06-26,i.ytimg i.ytimg yandex googleads.g.doubleclick...,34056.0,146,11,76,67,57,...,2,1,1,i i yandex g i mds tpc i mds g vk ads yandex g...,Email Email Search_Engines Email Broadcasts ...,0,0,0,1,0
363854,317716,2021-06-19,2021-08-12,instagram l.instagram vk yastatic google yande...,52898.0,452,30,821,342,312,...,4,3,1,instagram l vk yastatic google yandex ads appl...,community Other MobileApps Search_Engines C...,0,0,0,1,0
290394,414511,2021-06-16,2021-08-08,icloud m.love m.vk mail.yandex mail.yandex ssp...,57805.0,1449,396,1153,836,522,...,1,1,1,icloud m m mail mail ssp sunuserapi yandex yas...,Hosting Email Email Building_Supplies Searc...,0,0,0,1,0
225792,181270,2021-06-17,2021-07-17,ad.mail sunuserapi tpc.googlesyndication vk go...,20986.0,119,5,192,179,126,...,1,1,1,ad sunuserapi tpc vk g mail my cdn-rtb g mail ...,Other Email Email Hosting Email Other ...,0,0,0,1,0


## Объединение данных

Вычленяем из основного ДФ две выборки:
- all_data - c признаками и целевой переменной
- closed_data - без целевой переменной с идентификаторами пользователей из заданного для предсказания списка

In [ ]:
#Join data with target (in result -> train data)
all_data = target_data.set_index('user_id').join(all_users.set_index('user_id'))

#Join data with for closed users
closed_data = id_to_submit.set_index('user_id').join(all_users.set_index('user_id'))

del all_users

### Выделение только тех URL, которые есть в closed_data

Одна из идей, которая была попробована, чтобы повысить качество предсказания была достаточно читерской. Было сделано предположение, что преддсказания имеют низкое качество еще и из-за того, что модель учится на большом объеме сайтов, которые не посещали пользователи из закрытой выборки. А значит в процессе предсказания она сталкивается с признаками, для которых нет правил в модели. Чтобы избежать этого было предложено следующее:
- составлен список сайтов из закрытой выборки
- в списках для пользователей оставлены только те сайты, которые есть списке из закрытой выборки

Этот подход не дал ожидаемого результата и поэтому его реализация перемещена в текстовое поле.



```
closed_list =set((" ".join(closed_data.url_host)).split())
all_data.url_host = all_data.url_host.transform(lambda x: search_closed(x, closed_list))
```




## Обработка тренировочной выборки

Здесь размещены операции, которые по разным причинам мы применяем только к тренировочной выборке:
- удаление новых пользователей, по которым крайнемало статистики (из закрытой выборки мы не можем никого удолять, ведь мы должны дать предсказания по всем пользователям)
- расчет возрастной группы для каждого пользователя (естественно, в закрытой выборке нет данных по возрастам)
- удаление из выборки записей, в которых нет данных о поле

*Примечание:*

Если бы было больше времени, можно было бы:
- создавать отдельную тренировочную выборку для обучения модели расчета пола и отдельную выборку для обучения модели расчета возрастной группы)
- тогда из выборки для возрастов записи с нулевым полом можно было бы не удалять

In [ ]:
#Drop users who have the activity period is less than 2 days
all_data['period'] = all_data.end_date - all_data.begin_date
all_data['period'] = all_data['period'].astype('timedelta64[D]')
all_data = all_data.drop(['begin_date','end_date'], axis = 1)
all_data = all_data.query("period > 2")

In [ ]:
#Additing new column with age-group
all_data['klass'] = all_data.age.apply(my_class)

In [ ]:
#Drop rows with NA in target 
all_data.is_male = all_data.is_male.fillna('NA')
all_data = all_data.query('is_male != "NA"')
all_data.is_male = all_data.is_male.astype('int')

all_data = all_data.query('klass != 7')

In [ ]:
#Распределение по возрастным группам
all_data.klass.value_counts()

2    78512
3    69250
4    37507
1    29189
5    21075
6     3752
Name: klass, dtype: int64

## Расчет дополнительных признаков

### Расчет списка релевантных URL-s каждого пола
Идея в следующем: 
- найти список самых популярных сайтов для мужчин
- найти список самых популярных сайтов для женщин
- выделить те сайты, которые находять вне пересечения этих множеств и взять из них 10 популярных
- для этих популярных и уникальных сайтов составить доп. столбцы и посчитать количество их посещений каждым пользователем

Аналогично сделать для возрастных групп.
Таким образом мы бы создали в ДФ сайты, активное использование которыых должно было бы свидетельствовать к определенной возрастной группе/полу.

*Примечание:*

Если бы было больше времени, можно было бы:
- создавать отдельную тренировочную выборку для обучения модели расчета пола и отдельную выборку для обучения модели расчета возрастной группы)
- тогда столбцы были бы расчитаны один раз (для пола - по полу, для возраста - по возрастным группам)


In [ ]:
#Расчет списка популярных релевантных сайтом для мужчин и женщин
cat_for_features_male = search_commons(all_data, 355, 10)
cat_for_features_male

['Russian_Cars', 'Registrars', 'Viruses', 'Sex_shops', 'Logos', 'Offshores', 'Autotravel', 'Photobanks']
['Latina', 'Land_Cruisers', 'Belarusian_Literature', 'Database', 'UFO']


['Russian_Cars',
 'Registrars',
 'Viruses',
 'Sex_shops',
 'Logos',
 'Offshores',
 'Autotravel',
 'Photobanks',
 'Latina',
 'Land_Cruisers',
 'Belarusian_Literature',
 'Database',
 'UFO']

### Расчет списка релевантных категорий URL-s каждого возраста


In [ ]:
#Расчет списка популярных релевантных сайтом для возрастных групп
cat_for_features_age = search_commons_age(all_data, 355, 10)
cat_for_features_age

  0%|          | 0/7 [00:00<?, ?it/s]

['Animation', 'Unacknowledged', 'Managment', 'Country_Realty', 'Automation', 'Legal_Advice', 'Fantasy', 'Showtimes', 'Delivery', 'Parents']
['Caricatures', 'Country_Realty', 'Furniture', 'Managment', 'Parents', 'Stations', 'Automation', 'Legal_Advice', 'Catalogues', 'Online_shops']
['Cottage', 'Country_Realty', 'Stations', 'Furniture', 'Automation', 'Legal_Advice', 'Parents', 'Law', 'Catalogues', 'Managment']
['Stations', 'Country_Realty', 'Automation', 'realty_base', 'Furniture', 'Catalogues', 'Law', 'Legal_Advice', 'Java_games', 'Aviation']
['realty_base', 'Stations', 'Law', 'Java_games', 'Automation', 'Maps', 'Country_Realty', 'Furniture', 'Legal_Advice', 'Aviation']
['Eurovision', 'Womens_Life', 'Maps', 'Stations', 'Automation', 'Used', 'realty_base', 'Celebrations', 'Law', 'Java_games']
[]


['Animation',
 'Unacknowledged',
 'Managment',
 'Country_Realty',
 'Automation',
 'Legal_Advice',
 'Fantasy',
 'Showtimes',
 'Delivery',
 'Parents',
 'Caricatures',
 'Country_Realty',
 'Furniture',
 'Managment',
 'Parents',
 'Stations',
 'Automation',
 'Legal_Advice',
 'Catalogues',
 'Online_shops',
 'Cottage',
 'Country_Realty',
 'Stations',
 'Furniture',
 'Automation',
 'Legal_Advice',
 'Parents',
 'Law',
 'Catalogues',
 'Managment',
 'Stations',
 'Country_Realty',
 'Automation',
 'realty_base',
 'Furniture',
 'Catalogues',
 'Law',
 'Legal_Advice',
 'Java_games',
 'Aviation',
 'realty_base',
 'Stations',
 'Law',
 'Java_games',
 'Automation',
 'Maps',
 'Country_Realty',
 'Furniture',
 'Legal_Advice',
 'Aviation',
 'Eurovision',
 'Womens_Life',
 'Maps',
 'Stations',
 'Automation',
 'Used',
 'realty_base',
 'Celebrations',
 'Law',
 'Java_games']

### Сохранение релевантных URL в отдельный ДФ (чтобы можно было не повторять расчет)

In [ ]:
top_cat_for_agg = list(set(cat_for_features_male + cat_for_features_age))
top_cat_for_agg_df = pd.DataFrame({'top_cat_for_agg': top_cat_for_agg}) 
top_cat_for_agg_df.to_csv('top_cat_for_agg.csv')
top_cat_for_agg

['Logos',
 'Unacknowledged',
 'Catalogues',
 'Sex_shops',
 'Managment',
 'Fantasy',
 'Autotravel',
 'Celebrations',
 'Maps',
 'Legal_Advice',
 'Aviation',
 'Cottage',
 'realty_base',
 'Country_Realty',
 'Latina',
 'Registrars',
 'Parents',
 'Online_shops',
 'Belarusian_Literature',
 'Automation',
 'Russian_Cars',
 'Delivery',
 'Animation',
 'Stations',
 'Furniture',
 'Showtimes',
 'Used',
 'Offshores',
 'Land_Cruisers',
 'Law',
 'Java_games',
 'Database',
 'UFO',
 'Caricatures',
 'Photobanks',
 'Eurovision',
 'Womens_Life',
 'Viruses']

## Расчет статистики по релевантным URL-столбцам

In [ ]:
#Расчет числа посещений пользователей по каждому доп. столбцу с релевантным URL 
for cat in tqdm(top_cat_for_agg): 
  all_data[cat] = all_data.url_cat.transform(lambda x: Counter(str(x).split())[cat])
  closed_data[cat] = closed_data.url_cat.transform(lambda x: Counter(str(x).split())[cat])


  0%|          | 0/38 [00:00<?, ?it/s]

### Постъобработка

In [ ]:
#Удаление промежуточных столбцов
all_data = all_data.drop(['clean_url_host', 'url_cat'], axis = 1)
closed_data = closed_data.drop(['clean_url_host', 'url_cat'], axis = 1)

In [ ]:
#Проверка пропусков
all_data.isna().sum()

age               0
is_male           0
url_host          0
price          5687
evening           0
               ... 
Caricatures       0
Photobanks        0
Eurovision        0
Womens_Life       0
Viruses           0
Length: 66, dtype: int64

## Подготовка к ML

### Объявление списков признаков и целевых переменных

In [ ]:
#Prepare lists with names of features 
common_col = ['user_id']
tar_col = ['age', 'is_male', 'klass']
feat_col = list(all_data.columns)
for element in common_col + tar_col:
  try:
    feat_col.remove(element)
  except:
    print(element)

user_id


## Разделение открытых данных (тренировочных) на тренировочные и тестовые

Так как мы подбираем гиперпараметры через кроссвалидацию, а проверяем нашу модель на площадке организаторов по закрытым данным, то после того, как общая концепция решения была выбрана, размер тестовой выборки был минимизирован. Так как фактически, она нам вообще была не нужна.

In [ ]:
#Split in train and test samples
train_f, test_f, train_t, test_t = \
train_test_split(all_data[feat_col], all_data[tar_col], \
                 test_size = 0.00002, shuffle = True, stratify = all_data.is_male)

#Clear memory
del all_data

## Расчет пола

Расчет пола реализован через паплайн на базе GridSearchCV, обеспечивающего перебор гиперпараметров для двух шагов паплайна:
- расчета tf-idf матрицы по столбцу со списками сайтов
- классификатора на базе LGBMClassifier 

Также пробовался вариант использования метода понижения размерности методом PCA. На тренировочной выборке он давал потрясающие результаты в виде ускорения обучения (в разы) и увеличения качества предсказаний. Но на тестовой выборке результаты были слабыми. От него пришлось отказаться

In [ ]:
# # Считывание предъобученной модели
#pipe_dm = joblib.load('pipe_dm_nor.pkl')

# tfidf-pipe 
url_pipe = Pipeline([
    ('selector', ColumnSelector('url_host')),
    ('tfidf', TfidfVectorizer(min_df = 4, ngram_range=(1,3)))])

## complecs pipe 
pipe_dm = Pipeline(steps=[\
                       ('tfidf_url', url_pipe),
                       ('model', LGBMClassifier(random_state = 42, n_estimators = 1000, max_depth = -1, learning_rate = 0.07))\
                       ])
pipe_dm.fit(train_f, train_t.is_male)
print('pipe fit finished')

#Расчет пола
train_f['male_proba'] = pipe_dm.predict_proba(train_f)[:, 1]
test_f['male_proba'] = pipe_dm.predict_proba(test_f)[:, 1]
print('pipe predict finished')

#Промежуточные метрики
roc_auc_male_train_1 = roc_auc_score(train_t.is_male, train_f['male_proba'])
roc_auc_male_test_1 = roc_auc_score(test_t.is_male, test_f['male_proba'])
print(roc_auc_male_train_1, roc_auc_male_test_1)

#Сохранение модели
joblib.dump(pipe_dm, 'pipe_dm_nor.pkl')

<class 'pandas.core.series.Series'>
pipe fit finished
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
pipe predict finished
0.8508104693981058 1.0


['pipe_dm_nor.pkl']

## Доп признак для возраста

Расчет возраста реализован через паплайн на базе GridSearchCV, обеспечивающего перебор гиперпараметров для двух шагов паплайна:
- расчета tf-idf матрицы по столбцу со списками сайтов
- классификатора на базе LGBMClassifier 

Также пробовался вариант использования метода понижения размерности методом PCA. На тренировочной выборке он давал потрясающие результаты в виде ускорения обучения (в разы) и увеличения качества предсказаний. Но на тестовой выборке результаты были слабыми. От него пришлось отказаться.

Кроме того вместо классификатора предпринималась попытка использования регрессора для расчета точного возраста с последующим присвоением ему возрастной категории. Но у такого подхода метрики были хуже

In [ ]:
# tfidf-pipe 
url_pipe = Pipeline([
    ('selector', ColumnSelector('url_host')),
    ('tfidf', TfidfVectorizer(min_df = 3, ngram_range=(1,3)))])

## complecs pipe 
pipe_da = Pipeline(steps=[\
                       ('tfidf_url', url_pipe),
                       ('model', LGBMClassifier(n_estimators = 1000, num_leaves = 35, objective = 'multiclass', num_class = 6,\
                         max_depth = -1, learning_rate = 0.07, silent = False))\
                       ])

pipe_da.fit(train_f, train_t.klass)
print('model fit finished')

#Расчет возрастных групп
train_age_klass = pipe_da.predict(train_f)
test_age_klass = pipe_da.predict(test_f)
print('model predict finished')

#Промежуточные метрики
train_f1_2 = f1_score(train_t.klass, train_age_klass, average = 'weighted')
test_f1_2 = f1_score(test_t.klass, test_age_klass, average = 'weighted')
print(train_f1_2, test_f1_2)

#Сохранение модели
joblib.dump(pipe_da, 'model_da.pkl')

0.8122242314918986 0.4


['model_da.pkl']

In [ ]:
#Итоговые метрики
train_result_score_2 = 2*train_f1_2 + 2*roc_auc_male_train_1 - 1
test_result_score_2 = 2*test_f1_2 + 2*roc_auc_male_test_1 - 1
print(train_result_score_2, test_result_score_2)

2.3260694017800088 1.7999999999999998


# РАБОТА С ЗАКРЫТЫМИ ДАННЫМИ

## Предъобработка

In [ ]:
closed_data['period'] = closed_data.end_date - closed_data.begin_date
closed_data['period'] = closed_data['period'].astype('timedelta64[D]')

## Предсказание целевых переменных

In [ ]:
#Calc male
closed_male = pipe_dm.predict_proba(closed_data)[:, 1]

#Calc age group
closed_data_age = pipe_da.predict(closed_data)

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


## Фиксация результатов

In [ ]:
# ДФ с расчетами
results = pd.DataFrame({'is_male': closed_male, 'age':  closed_data_age}, index = closed_data.index)

#Уточнение типов данных
results = results.astype({'is_male': 'float64'})
results = results.astype({'age': 'int64'})

#Запись в файл
results.to_csv('mts_27.csv')

## Выводы

Целью соревнования было предсказать для каждого пользователя из закрытой выборки две целевые переменные (пол и возрастную группу). В качестве данных для обучения был предоставлен набор паркетных файлов с данными о запросах, поступающих с сайтов, которые посещали пользователи, содержащих:
- название сайта
- дата посещения
- время посещения (утро, день, вечер, ночь)
- данные устройства, с которого поступила информация (тип, производитель, модель, цена)
- область
- город.

Решение задания разбито на два блокнота:
1. В первом реализовано начальная обработка данных:
- считывание паркетных файлов
- агрегация записей по идентификаторам пользователей с расчетом агрегированных признаков по каждому пользователю:
- списка всех названий сайтов, размещенных в хронологической последовательности;
- количества посещений по времени суток (утром, днем, вечером, ночью)
- количества посещений по дням недели (понедельник, вторник и т.д.)
- наиболее часто встречающегося города
- наиболее часто встречающегося региона
- количества уникальных городов
- количества уникальных регионов
- наиболее часто встречающегося типа устройства
- наиболее часто встречающейся модели устройства
- средней цены устройства.

2. Во втором ноутбуке реализованы:
- базовая преодобработка (уточнение типов данных, обработка пропусков)
- дизайн новых признаков, связанных с названиями сайтов
- разделение выборки на закрытую и открытую, кторая в свою очередь поделена на тренироввочную и тестовую
- подбор гиперпараметров для двух моделей на базе LGBMClassifier, которые по отдельности должны рассчитывать целевые переменные
- обучение моделей и расчет целевых переменных для закрытых данных
- оформление расчетов и их выгрузка в файл, который можно отправить на площадку для submit-а.

В процессе дизайна реализованы и опробованы разные подходы к повышению информативности данных и снижению их размерности. Основные из них:
1. Удаление из названий сайтов доменных зон, чисел и повторяющихся комбинаций типа (подход показал свою эфективность и оставлен в решении)
2. Замена названий сайтов на их категории (автотовары, спорт, игры и т.д.), рассчитанные на основании отдельного общедоступного датафрейма, в котором храняться записи с названиями сайтов и их категориями с несколькими уровнями конкретизации (не дал заметного эффекта, в конечном варианте решения не применялся)
3. Создание дополнительных столбцов для каждого из популярных релевантных сайтов (*) с последующим расчетом числа посещений этих сайтов для каждого из пользователей

(*) популярные релевантные сайты по полу получаются так:
- рассчитывается список самых популярных сайтов для мужчин
- рассчитывается список самых популярных сайтов для женщин
- из списков выбираются те сайты, которые находять вне пересечения полученных множеств, из них сохраняются 10 самых популярных
- для этих популярных и уникальных сайтов составляются доп. столбцы и расчитывается количество посещений сайта столбца каждым пользователем
(**) популярные релевантные сайты по возрастной группе получаются аналогично, но стратификация проводиться по возрастной группе

4. Создание дополнительных столбцов для каждой из популярных релевантных категорий сайтов (*) с последующим расчетом числа посещений этих категорий для каждого из пользователей (подход не оправдал себя и в конечное решение не попал)

В результате моделирования была достигнута итоговая метрика, равная 1.62, рассчитанная по формуле:

метрика
```
score = 2*f1 + 2*roc_auc - 1, 
где:
- f1 - метрика для задачи классификации возрастных групп (f1_score(y_true, y_pred, average = 'weighted'))
- roc_auc - метрика для задачи классификации по полу (roc_auc_score(y_true, y_pred)
```

 Что можно сделать, чтобы стало лучше:
 1. Добавить новые признаки на этапе агрегации.
 1. Использовать CatBoost вместо LightGBM (к сожалению, скромные вычислительные мощности этому препятствовали)
 2. Не кодировать категориальные переменные, доверить это бустинговой модели (LightGBM и CatBoost умеют рабоать с ними напрямую).
 3. Подготавливать отдельные выборки с отдельными признаками (особенно по релевантным сайтам) для модели классификации по полу и модели классификации по возрастной группе.
 4. Подружить название сайта и время его посещения и создать новый столбец со списками сайтов, состоящими из пар (время, название сайта). 
 5. Использовать нейросети для преобработки тектов и генерирования признаков, на которых затем обучить бустинговую модель.







- 
